In [0]:
(spark.read.json("dbfs/mnt/pokemon/pokemon").createOrReplaceTempView("pokemon"))

In [0]:

query = """"
SELECT ingestion_date,   -- Seleciona a data de ingestão dos dados
       poke.*            -- Seleciona todas as colunas da estrutura JSON explodida
FROM pokemon
LATERAL VIEW EXPLODE(results) AS poke  -- Expande a coluna 'results', que contém uma lista de Pokémon, transformando cada item em uma nova linha
QUALIFY ROW_NUMBER() OVER (PARTITION BY poke.name ORDER BY ingestion_date DESC) = 1;  """"
## Mantém apenas a entrada mais recente para cada Pokémon, garantindo que não haja duplicatas baseadas no nome do Pokémon

##QUALIFY filtra os resultados para manter apenas a entrada mais recente de cada Pokémon.

df.spark.sql(query)coalesce(1).write.mode("overwrite").saveAsTable("pokemon_exploded")